# Text Analysis with Pandas
We've been __[scraping data with Python](http://github.com/hsarka/instagramScraper)__ for about two weeks and as Senate and Municipal Elections were held during that time, we decided to make a word cloud out of posts' captions.
<br> Before one can make a word cloud out of scraped data in Tableau, some data wrangling is required. Words need to be tokenized, stripped of special characters, their occurrence counted - stop words excluded - so the output file lines comprise of a word and its occurrence count.
<br> Our data, scraped with a Python script we wrote earlier, were in a TSV file. This Pandas piece of code reads the TSV file, picks only one - the latest - post instance, strips and splits the post caption and counts the occurance of each word. This list is then exported into an Excel file that could be used for making a Tableau Vizz __[(this word cloud visualization)](https://public.tableau.com/profile/sarka4960#!/vizhome/MunicipalElectionsWordCloud/WordCloud)__.

This is what the final word cloud looks like:

<img src="wordcloudInstaCzech.png">

***
## Importing Pandas
In this text analysis, Pandas is the only library we will need.

In [2]:
import pandas as pd

## Opening TSV File
When opening TSV file, Pandas reads it as a CSV with a \t delimiter. 
Also there's no header in our data.

In [3]:
sourceData = pd.read_csv("dumpedPostData.tsv.gz", compression='gzip', delimiter="\t", header = None, encoding = "utf-8")

In [4]:
sourceData.head()

,0,1,2,3,4,5,6
0,2018-09-29 14:30:02.217003,choco_afro,1879051365051152095,2018-09-29 11:26:39,Asistentka Natalie podávající ruku,9,1935
1,2018-09-29 14:30:02.217016,choco_afro,1878496704661368275,2018-09-28 17:04:39,Zastupitelé dále volí obecní radu (ta může být...,14,1788
2,2018-09-29 14:30:02.217062,choco_afro,1878495337804759224,2018-09-28 17:01:56,Komunální volby #1\nSlíbil jsem shrnout pár zá...,5,1414
3,2018-09-29 14:30:02.217127,choco_afro,1878447225010837769,2018-09-28 15:26:20,Fräulein Franke ve svém domovském prostředí,18,5644
4,2018-09-29 14:30:02.217134,choco_afro,1877008104522273019,2018-09-26 15:47:04,Studentka z Českých Budějovic na dnešní prohlí...,23,3059


## Renaming Columns
Rename the columns to make them more user-friendly.

In [5]:
sourceData.columns = ["scraping date", "username", "post id", "post date", "post text", "# of comments", "# number of likes"]
sourceData.head()

,scraping date,username,post id,post date,post text,# of comments,# number of likes
0,2018-09-29 14:30:02.217003,choco_afro,1879051365051152095,2018-09-29 11:26:39,Asistentka Natalie podávající ruku,9,1935
1,2018-09-29 14:30:02.217016,choco_afro,1878496704661368275,2018-09-28 17:04:39,Zastupitelé dále volí obecní radu (ta může být...,14,1788
2,2018-09-29 14:30:02.217062,choco_afro,1878495337804759224,2018-09-28 17:01:56,Komunální volby #1\nSlíbil jsem shrnout pár zá...,5,1414
3,2018-09-29 14:30:02.217127,choco_afro,1878447225010837769,2018-09-28 15:26:20,Fräulein Franke ve svém domovském prostředí,18,5644
4,2018-09-29 14:30:02.217134,choco_afro,1877008104522273019,2018-09-26 15:47:04,Studentka z Českých Budějovic na dnešní prohlí...,23,3059


## Finding the Newest Post Instance
### Ranking
To find the newest post instance in our data, we used grouping by _username_ and _post id_ as those are the unique identificators. We rank them descendingly by _scraping date_.

In [6]:
sourceData["rank"] = sourceData.groupby(by = ["username", "post id"])["scraping date"].transform(lambda x: x.rank(ascending = False))

Checking the assigned rank for a single post.

In [7]:
sourceData.loc[sourceData["post id"] == 1879051365051152095].head()

,scraping date,username,post id,post date,post text,# of comments,# number of likes,rank
0,2018-09-29 14:30:02.217003,choco_afro,1879051365051152095,2018-09-29 11:26:39,Asistentka Natalie podávající ruku,9,1935,113.0
621,2018-09-29 15:30:02.761334,choco_afro,1879051365051152095,2018-09-29 11:26:39,Asistentka Natalie podávající ruku,11,2115,112.0
1241,2018-09-29 16:30:02.150157,choco_afro,1879051365051152095,2018-09-29 11:26:39,Asistentka Natalie podávající ruku,13,2252,111.0
1861,2018-09-29 17:30:02.251014,choco_afro,1879051365051152095,2018-09-29 11:26:39,Asistentka Natalie podávající ruku,14,2367,110.0
2481,2018-09-29 18:30:02.603931,choco_afro,1879051365051152095,2018-09-29 11:26:39,Asistentka Natalie podávající ruku,14,2458,109.0


### The Newest Instance
We picked just the post ranked as # 1 to have the newest instance.

In [8]:
cleanData = sourceData.loc[sourceData["rank"] == 1]

In [9]:
cleanData.head()

,scraping date,username,post id,post date,post text,# of comments,# number of likes,rank
11,2018-09-29 14:30:02.217224,choco_afro,1874675251562179159,2018-09-23 10:32:06,Přesně před 80 lety byla vyhlášena všeobecná m...,30,2832,1.0
143,2018-09-29 14:30:08.431512,ods.cz,1871017608742297509,2018-09-18 09:25:01,Skvělá atmosféra v Olomouci. Děkujeme!,0,132,1.0
535,2018-09-29 14:30:31.021692,alenaschillerova,1868839511421899772,2018-09-15 09:17:32,Dny NATO v Ostravě,0,31,1.0
643,2018-09-29 15:30:03.222565,tomio.cz,1874913619066496354,2018-09-23 18:25:42,Sto korun za hodinu – hnutí STAN „demokratů“ F...,11,123,1.0
2407,2018-09-29 17:30:31.306534,strakovka,1820375570215909100,2018-07-10 12:28:20,Poslední červnový den – Francie – Elysejský pa...,1,36,1.0


## Data Cleansing
### New Data Frame for Text Analysis
To strip and split the text, we prepared a new DF with two columns - _original_ text and _clean_ text that will later contain a list of words.

In [20]:
newDataset = pd.DataFrame(cleanData['post text'])
newData = pd.concat([newDataset["post text"], newDataset["post text"]], axis=1, keys=['original', 'clean'])
newData.head()

,original,clean
11,Přesně před 80 lety byla vyhlášena všeobecná m...,Přesně před 80 lety byla vyhlášena všeobecná m...
143,Skvělá atmosféra v Olomouci. Děkujeme!,Skvělá atmosféra v Olomouci. Děkujeme!
535,Dny NATO v Ostravě,Dny NATO v Ostravě
643,Sto korun za hodinu – hnutí STAN „demokratů“ F...,Sto korun za hodinu – hnutí STAN „demokratů“ F...
2407,Poslední červnový den – Francie – Elysejský pa...,Poslední červnový den – Francie – Elysejský pa...


## Spliting with Regex
We split the words to a list by regex \W_ that includes characters a-z, A-Z, 0-9 and an underscore.

In [11]:
newData['clean'] = newData.clean.str.strip().str.split('[\W_]+')
newData.head()

,original,clean
11,Přesně před 80 lety byla vyhlášena všeobecná m...,"[Přesně, před, 80, lety, byla, vyhlášena, všeo..."
143,Skvělá atmosféra v Olomouci. Děkujeme!,"[Skvělá, atmosféra, v, Olomouci, Děkujeme, ]"
535,Dny NATO v Ostravě,"[Dny, NATO, v, Ostravě]"
643,Sto korun za hodinu – hnutí STAN „demokratů“ F...,"[Sto, korun, za, hodinu, hnutí, STAN, demokrat..."
2407,Poslední červnový den – Francie – Elysejský pa...,"[Poslední, červnový, den, Francie, Elysejský, ..."


## Pivoting List Items
Pivots the list to a word-per-line format.

In [12]:
rows = list()
for row in newData[['original', 'clean']].iterrows():
    r = row[1]
    for clean in r.clean:
        rows.append((r.original, clean))

clean = pd.DataFrame(rows, columns=['original', 'clean'])
clean.head(10)

,original,clean
0,Přesně před 80 lety byla vyhlášena všeobecná m...,Přesně
1,Přesně před 80 lety byla vyhlášena všeobecná m...,před
2,Přesně před 80 lety byla vyhlášena všeobecná m...,80
3,Přesně před 80 lety byla vyhlášena všeobecná m...,lety
4,Přesně před 80 lety byla vyhlášena všeobecná m...,byla
5,Přesně před 80 lety byla vyhlášena všeobecná m...,vyhlášena
6,Přesně před 80 lety byla vyhlášena všeobecná m...,všeobecná
7,Přesně před 80 lety byla vyhlášena všeobecná m...,mobilizace
8,Přesně před 80 lety byla vyhlášena všeobecná m...,
9,Skvělá atmosféra v Olomouci. Děkujeme!,Skvělá


## Getting Rid of Empty Strings
As we have some empty strings - remnants after splitting - we just filter these out.

In [13]:
clean = clean[clean.clean.str.len() > 0]
clean.head(10)

,original,clean
0,Přesně před 80 lety byla vyhlášena všeobecná m...,Přesně
1,Přesně před 80 lety byla vyhlášena všeobecná m...,před
2,Přesně před 80 lety byla vyhlášena všeobecná m...,80
3,Přesně před 80 lety byla vyhlášena všeobecná m...,lety
4,Přesně před 80 lety byla vyhlášena všeobecná m...,byla
5,Přesně před 80 lety byla vyhlášena všeobecná m...,vyhlášena
6,Přesně před 80 lety byla vyhlášena všeobecná m...,všeobecná
7,Přesně před 80 lety byla vyhlášena všeobecná m...,mobilizace
9,Skvělá atmosféra v Olomouci. Děkujeme!,Skvělá
10,Skvělá atmosféra v Olomouci. Děkujeme!,atmosféra


## Lowercasing
Lots of people play with upper/lowercase even on Instagram. For us from the meaning perspective, they are the same, so we decided to lowercase everything. (We'll deal with names manually afterwards.)

In [14]:
clean['clean'] = clean.clean.str.lower()
clean.head()
#clean.loc[clean["clean"]=="000"]
#df.loc[df['shield'] > 6]

,original,clean
0,Přesně před 80 lety byla vyhlášena všeobecná m...,přesně
1,Přesně před 80 lety byla vyhlášena všeobecná m...,před
2,Přesně před 80 lety byla vyhlášena všeobecná m...,80
3,Přesně před 80 lety byla vyhlášena všeobecná m...,lety
4,Přesně před 80 lety byla vyhlášena všeobecná m...,byla


## Counting
Here we count each word's occurance and reset the dataframe's index. The _word_ column will be used for stop-word lookup later.

In [21]:
counts = clean.clean.value_counts().to_frame()
counts = counts.reset_index()
counts.columns = ['word', 'occurance']
counts.head()

,word,occurance
0,a,951
1,v,677
2,na,613
3,se,514
4,je,335


## Stop words
For English stop words, you can use __[(library from NLTK)](https://www.nltk.org)__.
<br> As we're analysing Czech language, we use our own CSV file containing stop words. 

In [16]:
stop = pd.read_csv("stopwordsCzech.csv", header = None, encoding = "utf-8")
stop.columns = ["stopword"]
stop.head()

,stopword
0,a
1,aby
2,ačkoli
3,aj
4,ale


To compare the _word_ column and 'stopwords_czech.csv' file, we use __.isin()__. To exclude words that are stop-words, we use __.dropna()__. 
<br>We took the top 500 words for manual polishing afterwards with consideration of grammatical declension and conjugation.

In [24]:
countsStripped = counts[~counts["word"].isin(stop["stopword"])].dropna()
toExport = countsStripped.head(500)
toExport.head()

,word,occurance
22,spd,98
25,czech,79
26,volby,78
33,praha,66
37,2018,59


## Export to Excel
As Tableau works nicely with Excel files, we decided to export into this format. 

In [18]:
toExport.to_excel('wordcloudPrep.xlsx', sheet_name='words')